In [6]:
import torch 
from torch import nn

In [7]:
class ConvolutionBlock(nn.Module):
    def __init__(self,in_channel,out_channel,kernel_size,stride,padding):
        super().__init__()
        self.ConvoLayer_X1=nn.Conv2d(in_channel,out_channel,kernel_size,stride,padding)
        self.Relu_Layer=nn.ReLU()
        self.ConvoLayer_X2=nn.Conv2d(in_channel,out_channel,kernel_size,stride,padding)

    def forward(self,x):
        return self.ConvoLayer_X2(self.Relu_Layer(self.ConvoLayer_X1(x)))

In [8]:
class ResNetDefine(nn.Module):
    def __init__(self,in_channel,out_channel,first=False):
        super().__init__()

        if first:
            self.layer_1=nn.Conv2d(in_channel,out_channel,7,2,1)
            self.Relu_layer=nn.ReLU()

        self.conv_layer=ConvolutionBlock(in_channel,out_channel,3,1,0)


    def forward(self,x):
        if self.first:
            h=self.Relu_layer
        h=nn.ReLU(torch.add(x,h))

        return h


In [9]:
class DriverClass(nn.Module):
    def __init__(self):
        super().__init__()
        out_features=[ 64,128,256,512 ]
        classes=[ 13,27,74,19 ]
        self.Block=nn.ModuleList([ResNetDefine(3,64,True)])

        for i in range(len(out_features)):
            if i>0:
                self.Block.append(ResNetDefine(out_features[i-1],out_features[i]))
            for _ in range(2):
                self.Block.append(ResNetDefine(out_features[i],out_features[i]))

        self.avg_pool=nn.AvgPool2d(kernel_size=3)
        self.Full_connect_13=nn.Linear(512,13)

    def forward(self,x):
        for block in self.Block:
            x=block(x)

        x=self.avg_pool(x)
        x=torch.flatten(x)
        x=self.Full_connect_13(x)

        return x


In [11]:
if __name__ == "__main__":
    resnet=DriverClass()
    # data
    x=torch.rand(1,3,256,256)
    print(resnet(x).shape)

AttributeError: 'ResNetDefine' object has no attribute 'first'